In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data=pd.read_csv('/content/drive/MyDrive/iot-dataset/device_4_dataset.csv')


In [18]:
data.head()
data.groupby('type')['type'].count()

type
attack    923437
benign    175240
Name: type, dtype: int64

In [17]:
data["type"].replace({"gafgyt_combo": "attack" ,
                             "gafgyt_junk" : "attack",
                             "gafgyt_scan" : "attack",
                             "gafgyt_tcp" : "attack",
                             "gafgyt_udp" : "attack",
                             "mirai_ack" : "attack",
                             "mirai_scan" : "attack",
                             "mirai_syn" : "attack",
                             "mirai_udp" : "attack",
                             "mirai_udpplain" : "attack"},
                             inplace=True)

In [19]:
x = data.iloc[:,:-1].values
y = data.iloc[:,-1].values

In [20]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

In [ ]:
#standardize numerical columns
# def standardize(df,col):
#     df[col]= (df[col]-df[col].mean())/df[col].std()
# data_st=data.copy()
# for i in (data_st.iloc[:,:-1].columns):
#     standardize (data_st,i)
# data_st.head()

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2,shuffle=True)

In [22]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [23]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, matthews_corrcoef, f1_score, precision_score, recall_score

In [24]:
from keras import Sequential
from keras.layers import Dense

classifier = Sequential()

In [25]:
#First Hidden Layer
classifier.add(Dense(64, activation='relu', kernel_initializer='random_normal', input_dim=115))
#Second  Hidden Layer
classifier.add(Dense(32, activation='relu', kernel_initializer='random_normal'))

classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal'))

#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

In [26]:
#Compiling the neural network
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 64)                7424      
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dense_6 (Dense)             (None, 16)                528       
                                                                 
 dense_7 (Dense)             (None, 1)                 17        
                                                                 
Total params: 10,049
Trainable params: 10,049
Non-trainable params: 0
_________________________________________________________________


In [27]:
#Fitting the data to the training dataset
classifier.fit(X_train,y_train, validation_data=(X_test,y_test), epochs=1)

27467/27467 [==============================] - 67s 2ms/step - loss: 0.0027 - accuracy: 0.9994 - val_loss: 8.1875e-04 - val_accuracy: 0.9998


In [28]:
y_pred=classifier.predict(X_test)
y_pred =(y_pred>0.5)

In [29]:
print(f"Accuracy   : {accuracy_score(y_test, y_pred)}")
print(f"Precision  : {precision_score(y_test, y_pred)}")
print(f"F1-Measure : {f1_score(y_test, y_pred)}")
print(f"MCC        : {matthews_corrcoef(y_test, y_pred)}")
print(f"Recall     : {recall_score(y_test, y_pred)}")

Accuracy   : 0.999799759711654
Precision  : 0.9998564579565354
F1-Measure : 0.9993687230989957
MCC        : 0.9992498988197374
Recall     : 0.998881463848338


In [30]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    184869
           1       1.00      1.00      1.00     34867

    accuracy                           1.00    219736
   macro avg       1.00      1.00      1.00    219736
weighted avg       1.00      1.00      1.00    219736



In [31]:
print(confusion_matrix(y_test, y_pred))

[[184864      5]
 [    39  34828]]
